## Sample code to manually upload ML PY Framework data to Azure datastore in preparation for routing/transorming/training

### Note:  Intention is for input data (pre-route) to actually be pushed to Azure by payroll services (eg euHReka) 

## Imports

In [ ]:
import azureml.core
from azureml.core import Workspace
from ngamlfpy.pipeline import FileFinder
import os

# check core SDK version number
print("Azure ML SDK Version: ", azureml.core.VERSION)

## Workspace

In [ ]:
# load workspace configuration from the config.json file in the current folder.

ws = Workspace.from_config(path='azure_config_dev.json')
print(ws.name, ws.location, ws.resource_group, ws.location, sep='\t')

## Datastore

In [ ]:
ds = ws.get_default_datastore()
print(ds.datastore_type, ds.account_name, ds.container_name)

In [ ]:
ml_service = 'PAD'

## Upload input data (pre-route)

In [ ]:
source_finder = FileFinder(ml_service, use_model_name_in=False, base_folder = './sample_data',relative_input_folder = FileFinder.MLFOLDER_INPUT)
print('local source folder: ',source_finder.get_input_folder())
dest_finder = FileFinder(ml_service, use_model_name_in=False, relative_input_folder = FileFinder.MLFOLDER_INPUT)
print('azure dest folder: ',dest_finder.get_input_folder())

In [ ]:
file_names = []
for fname in os.listdir(source_finder.get_input_folder()):
    path = os.path.join(source_finder.get_input_folder(), fname)
    if os.path.isdir(path):
        # skip directories
        pass
    else:
       file_names.append(path) 
    
print(file_names)    
        

In [ ]:
ds.upload_files(file_names, target_path=dest_finder.get_input_folder(), overwrite=True, show_progress=True)

## Upload input data (post-route)

In [ ]:
ml_service = 'TWV'
model_name = 'SS01'
model_version = '011'

In [ ]:
source_finder = FileFinder(ml_service, use_model_name_in=True, model_name = model_name, model_version = model_version, relative_input_folder = FileFinder.MLFOLDER_INPUT)
print('local source folder: ',source_finder.get_input_folder())
dest_finder = FileFinder(ml_service, use_model_name_in=True, model_name = model_name, model_version = model_version, relative_input_folder = FileFinder.MLFOLDER_INPUT)
print('azure dest folder: ',dest_finder.get_input_folder())

In [ ]:

ds.upload(src_dir=source_finder.get_input_folder(), target_path=dest_finder.get_input_folder(), overwrite=True, show_progress=True)

## Upload train data (post-route and post-transform, ie training set ready for training)

In [ ]:
ml_service = 'PAD'
model_name =  'A001'
model_version= '001'

In [ ]:
source_finder = FileFinder(ml_service, use_model_name_in=True, model_name = model_name, model_version = model_version, relative_input_folder = FileFinder.MLFOLDER_TRANSFORMED)
print('local source folder: ',source_finder.get_input_folder())
dest_finder = FileFinder(ml_service, use_model_name_in=True, model_name = model_name, model_version = model_version, relative_input_folder = FileFinder.MLFOLDER_TRANSFORMED)
print('azure dest folder: ',dest_finder.get_input_folder())

In [ ]:
ds.upload(src_dir=source_finder.get_input_folder(),target_path=dest_finder.get_input_folder(), overwrite=True, show_progress=True)